In [61]:
import pandas as pd
import numpy as np

In [62]:
#load icr data:
ICRdata4: pd.DataFrame = pd.read_spss('data/ICR/ICR_Round4-Online+user+comments+responding+to+satire+and+news+videos_January+12,+2021_11.16 (1).sav')
ICRdata6: pd.DataFrame = pd.read_spss('data/ICR/ICR round 6.sav')
#only keep codable comments:
ICRdata4c = ICRdata4[ICRdata4['codable'] == 'Yes'].copy()
ICRdata6c = ICRdata6[ICRdata6['codable'] == 'Yes'].copy()

In [30]:
ICRdata4c.columns.to_list()

['Coder',
 'ID',
 'Mark_ID',
 'Anonymity',
 'Anonymity_9_TEXT',
 'codable',
 'Interaction',
 'Acknowledgement',
 'Topic_relevance',
 'Reasoning',
 'Opinion',
 'disagreement',
 'Ideologicaldirection',
 'Name_calling',
 'Vulgarity',
 'Attack_reputation',
 'Question_Intelligenc',
 'All_caps_function',
 'Sarcasm_to_criticize',
 'Individual_right',
 'discrimination',
 'Invoke_violence',
 'Tone',
 'Ideology_REC',
 'Namecalling_REC',
 'vulgar_REC',
 'attackreputation_REC',
 'questionIntelli_REC',
 'ALLCAPS_REC',
 'sarcasm_REC',
 'individualright_REC',
 'discrimination_REC',
 'invokeViolence',
 'PrimaryLast',
 'filter_$']

In [63]:
ICRdata6c.columns

Index(['StartDate', 'ID', 'Coder', 'Mark_ID', 'Anonymity', 'codable',
       'Interaction', 'Acknowledgement', 'TopicRelevance', 'Reasoning',
       'BackgroundInfo', 'Opinion', 'disagreement', 'Ideologicaldirection',
       'Name_calling', 'Vulgarity', 'Attack_reputation',
       'Question_Intelligenc', 'All_caps_function', 'Sarcasm_to_criticize',
       'Individual_right', 'discrimination', 'Invoke_violence', 'Tone'],
      dtype='object')

In [69]:
#get aggregate concept annotations for each coder and each comment:
#['INCIVILITY_DUMMY', 'INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY', '

# Define incivility list and other variables
incivility_list = ['Name_calling', 'Vulgarity', 'Attack_reputation', 'Question_Intelligenc', 
                   'All_caps_function', 'Sarcasm_to_criticize', 'Individual_right', 
                   'discrimination', 'Invoke_violence']
print("Incivility variables:", incivility_list)

Incivility variables: ['Name_calling', 'Vulgarity', 'Attack_reputation', 'Question_Intelligenc', 'All_caps_function', 'Sarcasm_to_criticize', 'Individual_right', 'discrimination', 'Invoke_violence']


In [81]:
#['INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY', 'LIBERAL_DUMMY', 'CONSERVATIVE_DUMMY']
interactivity_list = ['Acknowledgement']
rationality_list = [ 'Reasoning', 'BackgroundInfo'] #does this include 'Evidence'?? -> both Evidence and BackgroundInfo are used in the ICRdata6c dataset, but only Reasoning is used in the ICRdata4c dataset, and the codebook also includes 'evidence'
ICRdata4c['INTERACTIVITY_DUMMY'] = ICRdata4c['Acknowledgement'].apply(lambda x: 1 if str(x).startswith('Yes') else 0).astype(int)
ICRdata6c['INTERACTIVITY_DUMMY'] = ICRdata6c['Acknowledgement'].apply(lambda x: 1 if str(x).startswith('Yes') else 0).astype(int)


In [71]:
#create dummy variables for the political orientation of the comment:
ICRdata4c['LIBERAL_DUMMY']= ICRdata4c.Ideologicaldirection.apply(lambda x: 1 if x == 'Left/Liberal/Democratic' else 0)
ICRdata4c['CONSERVATIVE_DUMMY']= ICRdata4c.Ideologicaldirection.apply(lambda x: 1 if x == 'Right/Conservative/Republican' else 0)
ICRdata6c['LIBERAL_DUMMY']= ICRdata6c.Ideologicaldirection.apply(lambda x: 1 if x == 'Left/Liberal/Democratic' else 0)
ICRdata6c['CONSERVATIVE_DUMMY']= ICRdata6c.Ideologicaldirection.apply(lambda x: 1 if x == 'Right/Conservative/Republican' else 0)

In [12]:
ICRdata6c.loc[:, incivility_list].isna().sum()

Name_calling            0
Vulgarity               0
Attack_reputation       0
Question_Intelligenc    0
All_caps_function       0
Sarcasm_to_criticize    0
Individual_right        0
discrimination          0
Invoke_violence         0
dtype: int64

In [72]:
#replace any value starting with 'No' with 0 and else with 1:
for col in incivility_list:
    ICRdata6c.loc[:, col] = ICRdata6c.loc[:, col].apply(lambda x: 1 if str(x).startswith('Yes') else 0)


In [49]:
ICRdata6c.loc[:, incivility_list] = ICRdata6c.loc[:, incivility_list].astype(int)

In [73]:
#now create the same dummy variables as in ICRdata:
ICRdata6c.loc[:, 'INCIVILITY_DUMMY'] = (ICRdata6c.loc[:, incivility_list].sum(axis=1) > 0).astype(int)


In [74]:
#replace any value starting with 'No' with 0 and else with 1:
for col in rationality_list:
    ICRdata6c.loc[:, f'{col}_dum'] = (ICRdata6c.loc[:, col].apply(lambda x: 1 if str(x).startswith('Yes') else 0)).astype(int)
    print(f'{col}_dum created') 

Reasoning_dum created
BackgroundInfo_dum created


In [75]:
#now create the same dummy variables as in paper:
ICRdata6c.loc[:, 'RATIONALITY_DUMMY'] = (ICRdata6c.loc[:, ['Reasoning_dum', 'BackgroundInfo_dum']].sum(axis=1) > 0).astype(int)


In [ ]:
# Install required packages for inter-rater reliability
!pip install krippendorff scikit-learn

In [98]:
# Import required libraries for inter-rater reliability
import krippendorff
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Function to calculate Brennan-Prediger's Kappa
def brennan_prediger_kappa(rater1, rater2):
    """
    Calculate Brennan-Prediger's Kappa (also known as Kappa-BP)
    This assumes equal probability for each category (uniform marginals)
    """
    # Calculate observed agreement
    po = np.mean(rater1 == rater2)
    
    # Calculate expected agreement assuming uniform marginals
    # For binary coding (0,1), pe = 0.5
    n_categories = len(np.unique(np.concatenate([rater1, rater2])))
    pe = 1 / n_categories
    
    # Calculate Brennan-Prediger's Kappa
    if pe == 1:
        return np.nan
    else:
        kappa_bp = (po - pe) / (1 - pe)
    
    return kappa_bp

In [77]:
# Function to prepare data for inter-rater reliability using Mark_ID
def prepare_irr_data(data, variable, id_col='Mark_ID', coder_col='Coder'):
    """
    Prepare data for inter-rater reliability calculations
    Returns ratings from Coder 1 and Coder 2 for the same items (Mark_ID)
    """
    # Get data for each coder
    coder1_data = data[data[coder_col] == 'Coder 1'][[id_col, variable]].set_index(id_col)
    coder2_data = data[data[coder_col] == 'Coder 2'][[id_col, variable]].set_index(id_col)
    
    # Find common Mark_IDs coded by both coders
    common_ids = coder1_data.index.intersection(coder2_data.index)
    
    # Get ratings for common items
    coder1_ratings = coder1_data.loc[common_ids, variable].values
    coder2_ratings = coder2_data.loc[common_ids, variable].values
    
    print(f"Variable: {variable}")
    print(f"Number of items coded by both coders: {len(common_ids)}")
    print(f"Coder 1 distribution: {np.bincount(coder1_ratings)}")
    print(f"Coder 2 distribution: {np.bincount(coder2_ratings)}")
    
    return coder1_ratings, coder2_ratings, common_ids

In [99]:
# Function to calculate all inter-rater reliability metrics
def calculate_irr_metrics(coder1_ratings, coder2_ratings, variable_name):
    """
    Calculate Krippendorff's alpha, Brennan-Prediger's kappa, percent agreement, and prevalence
    """
    # Prepare data for Krippendorff's alpha (needs specific format)
    # Krippendorff expects a 2D array where each row is a coder and each column is an item
    kripp_data = np.array([coder1_ratings, coder2_ratings])
    
    # Calculate Krippendorff's Alpha
    try:
        alpha = krippendorff.alpha(kripp_data, level_of_measurement='nominal')
    except:
        alpha = np.nan
    
    # Calculate Brennan-Prediger's Kappa
    try:
        kappa_bp = brennan_prediger_kappa(coder1_ratings, coder2_ratings)
    except:
        kappa_bp = np.nan
    
    # Calculate Percent Agreement
    agreement = np.mean(coder1_ratings == coder2_ratings) * 100
    
    # Calculate Prevalence (proportion of positive cases)
    total_ratings = len(coder1_ratings) + len(coder2_ratings)
    positive_cases = np.sum(coder1_ratings) + np.sum(coder2_ratings)
    prevalence = (positive_cases / total_ratings) * 100
    
        
    return {
        'Variable': variable_name,
        'N_Items': len(coder1_ratings),
        'Krippendorff_Alpha': alpha,
        'Brennan_Prediger_Kappa': kappa_bp,
        'Percent_Agreement': agreement,
        'Occurrence (% of comments)': prevalence
    }

In [100]:
# Calculate inter-rater reliability for all specified variables
variables_to_analyze = ['INCIVILITY_DUMMY', 'INTERACTIVITY_DUMMY', 'RATIONALITY_DUMMY', 'LIBERAL_DUMMY', 'CONSERVATIVE_DUMMY']

# Store results
irr_results = []

print("=== Inter-Rater Reliability Analysis ===\n")

for variable in variables_to_analyze:
    print(f"\n--- {variable} ---")
    
    # Prepare data for this variable
    coder1_ratings, coder2_ratings, common_ids = prepare_irr_data(ICRdata6c, variable)
    
    # Calculate metrics
    metrics = calculate_irr_metrics(coder1_ratings, coder2_ratings, variable)
    irr_results.append(metrics)
    
    # Print results
    print(f"Krippendorff's Alpha: {metrics['Krippendorff_Alpha']:.4f}")
    print(f"Brennan-Prediger's Kappa: {metrics['Brennan_Prediger_Kappa']:.4f}")
    print(f"Percent Agreement: {metrics['Percent_Agreement']:.2f}%")
    print(f"Occurrence (% of comments): {metrics['Occurrence (% of comments)']:.2f}%")
    print("-" * 50)

=== Inter-Rater Reliability Analysis ===


--- INCIVILITY_DUMMY ---
Variable: INCIVILITY_DUMMY
Number of items coded by both coders: 196
Coder 1 distribution: [ 86 110]
Coder 2 distribution: [ 75 121]
Krippendorff's Alpha: 0.6741
Brennan-Prediger's Kappa: 0.6837
Percent Agreement: 84.18%
Occurrence (% of comments): 58.93%
--------------------------------------------------

--- INTERACTIVITY_DUMMY ---
Variable: INTERACTIVITY_DUMMY
Number of items coded by both coders: 196
Coder 1 distribution: [113  83]
Coder 2 distribution: [144  52]
Krippendorff's Alpha: 0.5605
Brennan-Prediger's Kappa: 0.6020
Percent Agreement: 80.10%
Occurrence (% of comments): 34.44%
--------------------------------------------------

--- RATIONALITY_DUMMY ---
Variable: RATIONALITY_DUMMY
Number of items coded by both coders: 196
Coder 1 distribution: [155  41]
Coder 2 distribution: [135  61]
Krippendorff's Alpha: 0.2333
Brennan-Prediger's Kappa: 0.4082
Percent Agreement: 70.41%
Occurrence (% of comments): 26.02%
--

In [101]:
# Create a summary table of all results
irr_summary = pd.DataFrame(irr_results)
print("\n=== SUMMARY TABLE ===")
print(irr_summary.round(2))


=== SUMMARY TABLE ===
              Variable  N_Items  Krippendorff_Alpha  Brennan_Prediger_Kappa  \
0     INCIVILITY_DUMMY      196                0.67                    0.68   
1  INTERACTIVITY_DUMMY      196                0.56                    0.60   
2    RATIONALITY_DUMMY      196                0.23                    0.41   
3        LIBERAL_DUMMY      196                0.69                    0.78   
4   CONSERVATIVE_DUMMY      196                0.76                    0.88   

   Percent_Agreement  Occurrence (% of comments)  
0              84.18                       58.93  
1              80.10                       34.44  
2              70.41                       26.02  
3              88.78                       23.47  
4              93.88                       15.31  
